In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [3]:
train

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,USA,12,103,161,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,USA,31,106,179,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,USA,17,103,168,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,USA,14,68,109,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         35452 non-null  int64  
 1   Q1            35333 non-null  float64
 2   Q2            35305 non-null  float64
 3   Q3            35322 non-null  float64
 4   Q4            35327 non-null  float64
 5   Q5            35362 non-null  float64
 6   Q6            35320 non-null  float64
 7   Q7            35310 non-null  float64
 8   Q8            35344 non-null  float64
 9   Q9            35348 non-null  float64
 10  Q10           35232 non-null  float64
 11  Q11           35333 non-null  float64
 12  Q12           35303 non-null  float64
 13  Q13           35356 non-null  float64
 14  Q14           35350 non-null  float64
 15  Q15           35345 non-null  float64
 16  Q16           35367 non-null  float64
 17  Q17           35373 non-null  float64
 18  Q18           35305 non-nu

In [6]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)
train_y = train['nerdiness']

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [8]:
train.isna().sum()

Q1               41
Q2               69
Q3               50
Q4               71
Q5               38
Q6               48
Q7               76
Q8               48
Q9               56
Q10              72
Q11              59
Q12              67
Q13              40
Q14              36
Q15              45
Q16              33
Q17              37
Q18              63
Q19              53
Q20              45
Q21              39
Q22              38
Q23              50
Q24              61
Q25              44
Q26              68
introelapse       0
testelapse        0
surveyelapse      0
TIPI1            53
TIPI2            66
TIPI3            79
TIPI4            64
TIPI5            70
TIPI6            62
TIPI7            64
TIPI8            65
TIPI9            64
TIPI10           80
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9              0
VCL10             0
VCL11             0


In [9]:
#train을 target과 feature로 나눠줍니다.
#train_ = train.dropna(axis=0)
train_= IterativeImputer().fit_transform(train.iloc[:, :-1])
train_= pd.DataFrame(train_)
test= IterativeImputer().fit_transform(test)
test= pd.DataFrame(test)
#train_ = train

In [10]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 67 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  float64
 1   1       15000 non-null  float64
 2   2       15000 non-null  float64
 3   3       15000 non-null  float64
 4   4       15000 non-null  float64
 5   5       15000 non-null  float64
 6   6       15000 non-null  float64
 7   7       15000 non-null  float64
 8   8       15000 non-null  float64
 9   9       15000 non-null  float64
 10  10      15000 non-null  float64
 11  11      15000 non-null  float64
 12  12      15000 non-null  float64
 13  13      15000 non-null  float64
 14  14      15000 non-null  float64
 15  15      15000 non-null  float64
 16  16      15000 non-null  float64
 17  17      15000 non-null  float64
 18  18      15000 non-null  float64
 19  19      15000 non-null  float64
 20  20      15000 non-null  float64
 21  21      15000 non-null  float64
 22

In [11]:
norm=Normalizer()
sc=StandardScaler()

train_S = norm.fit_transform(train_)
train_S = pd.DataFrame(train_S)
test= norm.transform(test) 
test = pd.DataFrame(test)
pca = PCA(n_components=50)
pca.fit(train_S)
#pca.fit(test)
train_pca = pca.transform(train_S)
test_pca = pca.transform(test)

In [12]:
train_pca.shape

(15000, 50)

In [13]:
test_pca.shape

(35452, 50)

In [14]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',
            'pca_component_31','pca_component_32','pca_component_33',
            'pca_component_34','pca_component_35','pca_component_36',
            'pca_component_37','pca_component_38','pca_component_39',
            'pca_component_40','pca_component_41','pca_component_42',
            'pca_component_43','pca_component_44','pca_component_45',
            'pca_component_46','pca_component_47','pca_component_48',
            'pca_component_49','pca_component_50']
trainDF_pca = pd.DataFrame(train_pca, columns=pca_columns)

In [15]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',
            'pca_component_31','pca_component_32','pca_component_33',
            'pca_component_34','pca_component_35','pca_component_36',
            'pca_component_37','pca_component_38','pca_component_39',
            'pca_component_40','pca_component_41','pca_component_42',
            'pca_component_43','pca_component_44','pca_component_45',
            'pca_component_46','pca_component_47','pca_component_48',
            'pca_component_49','pca_component_50']
testDF_pca = pd.DataFrame(test_pca, columns=pca_columns)

In [16]:
trainDF_pca

,pca_component_1,pca_component_2,pca_component_3,pca_component_4,pca_component_5,pca_component_6,pca_component_7,pca_component_8,pca_component_9,pca_component_10,pca_component_11,pca_component_12,pca_component_13,pca_component_14,pca_component_15,pca_component_16,pca_component_17,pca_component_18,pca_component_19,pca_component_20,pca_component_21,pca_component_22,pca_component_23,pca_component_24,pca_component_25,pca_component_26,pca_component_27,pca_component_28,pca_component_29,pca_component_30,pca_component_31,pca_component_32,pca_component_33,pca_component_34,pca_component_35,pca_component_36,pca_component_37,pca_component_38,pca_component_39,pca_component_40,pca_component_41,pca_component_42,pca_component_43,pca_component_44,pca_component_45,pca_component_46,pca_component_47,pca_component_48,pca_component_49,pca_component_50
0,0.077735,0.856428,0.232783,0.118540,-0.017670,0.012279,-0.003522,-0.002559,0.004214,0.002654,0.003743,-0.009618,-0.001636,0.000811,0.001870,0.003075,0.002038,0.006700,-0.003134,-0.001693,0.001365,-0.004129,-0.000196,0.001894,-0.003268,0.000098,-0.000382,-0.000365,0.002728,-0.002220,0.000363,-0.003705,-0.001593,0.000601,0.001378,-0.001008,-0.000421,-0.000567,-0.004045,-0.000476,0.005113,-0.002882,-0.000521,-0.001274,-4.390600e-03,-0.000514,-0.001035,0.000883,0.000706,0.000704
1,-0.186230,0.031118,-0.089013,0.168991,0.063741,-0.008921,-0.001299,0.018093,-0.002354,-0.005918,-0.011252,0.000202,-0.009731,0.009277,-0.013456,-0.008778,-0.007650,-0.005483,-0.004646,0.000020,0.000042,-0.009446,0.013443,-0.007390,-0.001473,-0.004664,-0.005532,0.000181,-0.005596,-0.011764,0.004305,-0.003233,-0.004718,-0.001092,-0.001930,-0.001203,-0.002221,0.005848,0.002265,0.001016,-0.002556,-0.002240,-0.000286,0.006080,4.677862e-03,0.001398,-0.001927,-0.000898,0.000008,-0.002229
2,-0.146147,0.223157,-0.097277,0.135067,0.034197,-0.014764,0.001375,0.000456,0.014738,0.013234,0.003720,-0.000095,0.007242,-0.012178,0.002763,0.015025,-0.014329,0.008992,0.005924,0.005523,0.000426,-0.010296,0.006949,0.004959,0.000544,0.003861,-0.013499,-0.005697,0.002399,0.001077,0.013546,-0.010298,0.006830,-0.003217,-0.005944,-0.000775,-0.006314,-0.001317,0.006223,0.001183,0.003135,-0.002631,0.000543,-0.001836,-5.332334e-04,0.001273,0.000239,0.001225,-0.003728,0.005959
3,-0.207722,0.132747,0.017749,-0.016346,-0.014813,-0.013812,0.006710,-0.008009,0.002629,-0.008597,-0.004412,-0.006655,0.001612,0.006723,0.004137,0.002309,0.005311,-0.002817,0.005732,-0.008581,-0.001354,-0.001062,-0.006731,0.001814,0.014269,-0.008176,0.005203,-0.004904,-0.004140,0.005287,0.001265,0.002349,-0.005341,-0.006142,0.001260,-0.001926,0.005144,-0.001767,0.003327,0.005240,-0.000087,0.004462,-0.001599,-0.003022,2.199857e-03,-0.000816,-0.000132,0.002481,-0.002331,-0.005119
4,-0.068960,0.628456,0.140027,-0.004311,0.012245,0.009329,-0.004302,-0.004074,-0.001022,-0.001615,0.005535,0.001956,-0.001873,-0.000352,0.001274,-0.000035,-0.001056,-0.000911,0.000590,0.000110,0.000139,-0.000247,-0.003753,-0.001716,-0.001145,0.000752,0.001671,-0.000213,0.000018,-0.000072,0.000724,-0.001460,0.000521,-0.000575,-0.000750,-0.000641,-0.001423,-0.002168,-0.000245,-0.000795,-0.000241,-0.000085,0.000638,0.000395,-1.346032e-07,-0.000696,-0.000163,0.001152,0.000636,0.000880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.176550,-0.016942,0.007016,-0.032279,-0.006203,-0.013056,0.000015,0.011909,-0.004218,0.003967,-0.004724,-0.015410,-0.009333,0.002187,-0.003223,-0.002351,-0.002742,0.006535,0.006834,0.005777,-0.003012,0.005525,0.003634,0.001402,-0.003393,0.003789,-0.002067,-0.001702,0.002708,0.000640,0.000411,0.003693,-0.001698,0.008124,0.000940,0.000199,0.001524,-0.005374,-0.000439,0.001058,-0.001873,-0.004366,0.001911,-0.000638,-3.825289e-03,-0.001577,-0.000998,-0.001398,-0.000884,-0.000957
14996,-0.105696,-0.057395,-0.082214,0.043394,0.04365

In [17]:
testDF_pca

,pca_component_1,pca_component_2,pca_component_3,pca_component_4,pca_component_5,pca_component_6,pca_component_7,pca_component_8,pca_component_9,pca_component_10,pca_component_11,pca_component_12,pca_component_13,pca_component_14,pca_component_15,pca_component_16,pca_component_17,pca_component_18,pca_component_19,pca_component_20,pca_component_21,pca_component_22,pca_component_23,pca_component_24,pca_component_25,pca_component_26,pca_component_27,pca_component_28,pca_component_29,pca_component_30,pca_component_31,pca_component_32,pca_component_33,pca_component_34,pca_component_35,pca_component_36,pca_component_37,pca_component_38,pca_component_39,pca_component_40,pca_component_41,pca_component_42,pca_component_43,pca_component_44,pca_component_45,pca_component_46,pca_component_47,pca_component_48,pca_component_49,pca_component_50
0,-0.197691,-0.141180,0.053990,-0.027623,-0.004641,-0.001835,-0.004041,0.004217,-0.000363,-0.008337,0.008350,0.002406,-0.000446,0.002569,-0.000898,0.002228,-0.000163,0.000898,-0.002852,0.004647,-0.002198,0.004911,-0.006256,-0.001504,0.005616,-0.000539,-0.003100,-0.001203,-0.001625,-0.002703,0.000201,-0.005621,-0.002392,0.001953,0.003747,-0.002606,-0.002010,-0.000292,-0.004372,0.001684,-0.002851,0.001967,-0.000586,-0.001951,-0.000133,-0.002381,0.001611,-0.000289,0.001535,0.000620
1,-0.164818,-0.105525,-0.012240,0.011382,0.011859,-0.015981,0.000873,0.001779,0.014399,-0.004036,0.008551,0.004805,0.000196,0.000702,-0.003215,-0.003413,-0.002030,0.003530,-0.004005,0.014818,0.004048,0.004726,0.000380,-0.000927,0.002954,0.001139,0.000280,0.001122,0.000916,0.002677,0.004741,0.001075,-0.000138,0.002514,0.000038,0.000308,-0.002485,0.000814,-0.001062,0.002974,0.001279,0.002209,0.000602,-0.000052,0.003267,0.000344,0.003214,-0.000854,-0.003519,-0.000953
2,-0.208208,-0.150183,0.059119,-0.019242,-0.033890,0.000071,-0.003767,0.014985,-0.010293,-0.011988,-0.002371,-0.000946,-0.004274,-0.001724,-0.000746,0.000888,-0.006006,0.010452,-0.006361,0.002948,-0.002833,-0.001644,0.001564,0.002927,-0.002022,0.003987,-0.006046,-0.002344,-0.001246,-0.001720,0.006994,-0.002558,0.002736,-0.000770,-0.000131,0.003962,0.002372,0.003040,0.001147,0.000880,-0.002815,0.001311,-0.004066,0.000032,-0.006532,0.002025,0.002987,0.000726,-0.002267,0.002875
3,1.093656,-0.037513,0.087732,0.048839,-0.003266,-0.001007,0.000211,-0.001382,-0.000152,0.000725,-0.000951,0.000309,-0.000071,0.000025,-0.000227,-0.000104,-0.000529,-0.000199,0.000007,0.000298,-0.000257,0.000182,0.000419,0.000425,0.000503,0.000271,-0.000153,0.000490,0.000135,0.000259,0.000010,-0.000236,-0.000314,0.000293,0.000699,-0.000003,0.000088,-0.000066,0.000301,0.000139,-0.000034,0.000566,-0.000135,0.000033,0.000130,-0.000361,0.000011,0.000004,-0.000143,0.000025
4,-0.167379,-0.072025,0.042109,-0.072880,0.003570,-0.002003,-0.007704,0.006067,-0.001406,0.000106,-0.004540,-0.000718,-0.009567,0.000952,0.000687,0.004025,0.000068,0.004862,-0.001291,0.002792,0.000868,0.005198,0.001507,-0.001218,-0.000289,-0.001944,-0.000627,0.001584,-0.000221,-0.000890,-0.002736,0.006371,0.000422,-0.002313,-0.000603,-0.001690,0.003591,-0.000984,-0.002608,0.003216,0.001778,0.002697,-0.000610,-0.000097,0.001103,0.004483,0.004224,0.001481,0.000446,0.000392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,-0.195362,0.083229,0.041981,-0.064125,-0.000998,0.015822,-0.001369,-0.002293,0.002925,-0.000163,0.002979,-0.000216,0.000270,0.001436,0.001361,-0.006029,0.006965,0.004369,-0.004504,0.002412,0.001327,-0.001257,0.005140,-0.003145,0.005428,0.001156,0.004401,0.000805,0.001823,-0.001152,0.003118,-0.001448,-0.001306,-0.001533,0.000403,-0.001526,0.002172,0.001436,0.000563,-0.001952,0.001824,0.001078,-0.003988,-0.000386,0.001406,0.002007,0.002137,0.006361,0.000846,-0.000830
35448,0.884915,0.006453,-0.044518,-0.048898,-0.004771,-0.004258,-0.001781,-0.000211,0.001092,-0.

In [18]:
y_target=train['nerdiness']

In [ ]:
start_time=time.time()
lgb= lgb.LGBMClassifier(
            n_estimators=1000
        )
parametes= {
    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]
}
clf=RandomizedSearchCV(lgb,parametes,scoring='roc_auc',n_iter=100)
clf.fit(X=trainDF_pca,y=y_target)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)

In [ ]:
# 재학습 위한 공간

In [ ]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = clf.predict(testDF_pca)

In [ ]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

In [ ]:
submission["nerdiness"] = lgbm_pred

In [ ]:
submission

In [ ]:
submission.to_csv("baseline_notNA-iterative-withNorm-PCA.csv", index = False)